# Тетрадь 2: Преобразователи данных

Предметом изучения в данной тетради являются преобразователи данных. Мы разберём, что они 
из себя представляют, как они устроены и для чего нужны.

## Содержание

- [Теория](#Теория)
...

Приступим теперь непосредственно к изучению материала.

***

## Теория

### Преобразование и предварительная обработка данных

**Предварительная обработка данных** (Data Preprocessing) в машинном обучении — это набор методов и 
техник, которые применяются к исходным данным перед их использованием для обучения модели. Цель 
предварительной обработки — подготовить данные к анализу, улучшить их качество и сделать их более 
пригодными для обучения моделей машинного обучения.

Вообще, существует множество различных этапов предварительной обработки данных. Однако в контексте 
данного курса мы будем относить всякие манипуляции, будь то очистка, сокращение размерности или 
форматирование, к преобразованиям данных. Отсюда **преобразователи данных** — программные интерфейсы 
(в основном классы), которые переводят данные из одного состояние в другое.

Проиллюстрируем такой этап предобработки, как дополнение данных. Суть дополнения в том, что 
информация, которую вы где-либо раздобыли для анализа далеко не всегда является полной (например, в 
таблице присутствуют пустые поля). Ниже представлена диаграмма, которая схематично описывает эффект 
применения так называемого `Imputer`'а к данным с пропусками.

<div style="text-align: center">
    <img src="../images/imputer_scheme.png" alt="Imputer Scheme" width=512 height=256>
</div>

Как можно заметить на диаграмме, в первой таблице (Raw Data) присутствовали некоторые пропущенные 
поля. После пропуска данных через `Imputer` всё стало на свои места. Столбцы были заполнены по 
соответствующим закономерностям.

### Некоторые этапы предварительной обработки данных

Возвращась к разнообразию этапов предварительной обработки данных, пречислим некоторые из них: 

- **Разделение:** тренировочный/тестовый/валидационный набор; кросс-валидация.

- **Очистка:** удаление дубликатов; обработка пропущенных значений; удаление шума.

- **Масштабирование:** нормализация/Стандартизация (например, Min-Max нормализация, 
Z-масштабирование); логарифмирование/экспоненцирование.

- **Кодирование:** бинарное кодирование (One-Hot Encoding); лэйбл-кодирование (Label Encoding).

- **Уменьшение Размерности:** PCA (Principal Component Analysis); t-SNE (t-distributed Stochastic 
Neighbor Embedding); LDA (Linear Discriminant Analysis).

### Порядок выполнения этапов предварительной обработки

Очень важно помнить, что порядок выполнения этих этапов действительно имеет огромное значение.
Сначала выполняется разделение данных, затем очистка, а потом уже всё остальное. Также далее мы 
обсудим не менее значимый нюанс касаемо применения преобразователей к отдельным наборам данных.

***

## Примеры

Рассмотрим теперь несколько упрощённых и один полный пример преобразователей данных. Начнём с 
более простых функций, а далее продемонстрируем полноценный класс. В качетве преобразования для 
примера возьмём Z-масштабирование. Формула для этой операции выглядит следующим образом:

$$Z(X) = \frac{X - \textrm{Mean}(X)}{\textrm{Std}(X)},$$

где
- $X$ — множество всех точек данных признака;
- $\textrm{Mean}(X)$ — среднее по $X$;
- $\textrm{Std}(X)$ — стандартное отклонение по $X$.

Запрограммируем функцию для Z-масштабирования.

In [2]:
import numpy as np
from numpy import ndarray

def z_scale(data: ndarray) -> ndarray:
    scaled = (data - np.mean(data)) / np.std(data)
    return scaled

Теперь представим некоторый случайный набор данных и применим к нему нашу функцию.

In [15]:
(size, round_to) = ((10, 4), 3)
selection = np.round(np.random.standard_normal(size), round_to)
scaled = np.round(z_scale(selection), round_to)

print(f"Before scaling:\n{selection}\n\nAfter scaling:\n{scaled}")

Before scaling:
[[-0.64  -0.555 -0.101  0.899]
 [-1.279 -0.949  0.066 -2.702]
 [ 0.425 -1.491 -1.077  1.915]
 [-0.956  0.737 -0.307  0.949]
 [-0.456  0.661  1.113  0.674]
 [-0.916  1.174  0.462  1.318]
 [-0.704 -1.318  0.706  0.032]
 [ 1.61  -0.799  0.934 -1.414]
 [-0.196  0.475  1.361 -0.563]
 [ 0.413 -0.166  0.948 -0.142]]

After scaling:
[[-0.644 -0.559 -0.105  0.896]
 [-1.283 -0.953  0.063 -2.707]
 [ 0.422 -1.495 -1.081  1.913]
 [-0.96   0.734 -0.311  0.946]
 [-0.46   0.658  1.11   0.671]
 [-0.92   1.171  0.459  1.315]
 [-0.708 -1.322  0.703  0.028]
 [ 1.607 -0.803  0.931 -1.418]
 [-0.2    0.472  1.358 -0.567]
 [ 0.41  -0.17   0.945 -0.146]]


Как видим, всё работает отлично! Однако, есть нюанс, о котором было упомянуто в конце теоретического 
раздела, и сейчас мы его обусдим.

Итак, представьте, что вы обучаете модель на тренировочных данных. Вы готовите её к взаимодействию 
с информацией, которую она ещё не видела. Но во время преобразования данных вы применяете инструмент 
трансформации к каждому из наборов (тренировочному и тестовому) по отдельности. То есть та же функция 
`z_scale()` принимает во внимание характеристики тестового набора данных при его масштабировании. 
Теперь представим, что тестового набора нет. Вам только предстоит его получить и обработать. 
Получается, для преобразования тестового набора данных нам нужно заглянуть в будущее!

Многие начинающие часто совершают эту ошибку. Новички трансформируют тестовые данные, пользуясь их 
характеристиками. Хоть это и явное противоречие, часто его никто не замечает. Так что же делать с 
этим противоречием? 

Для решения этой проблемы нам понадобится хранить параметры тренировочных данных (минимум, максимум, 
среднее, медиану, стандартное отклонение и т.п.) и использовать их для преобразований тестовых. 
Рассмотрим конкретную реализацию.

Для начала зададим интерфейс преобразователя в той форме, в которой нам будет удобно им пользоваться.
За основу возьмём те же классы-трансформеры из библиотеки Scikit-Learn.

In [16]:
from abc import ABC, abstractmethod
from typing import Any, Tuple

from numpy import ndarray


class BasePreprocessor(ABC):
    """The Base Preprocessor class is an abstract base class for preprocessor
    implementations.
    """

    def __init__(self, copy: bool = True) -> None:
        self.copy = copy

    @abstractmethod
    def fit(self, x) -> None:
        """Fit the preprocessor to the provided features."""
        message = "Every preprocessor must implement the `fit()` method."
        raise NotImplementedError(message)

    @abstractmethod
    def transform(self, x) -> Any:
        """Transform the input features."""
        message = (
            "Every preprocessor must implement the `transform()` method."
        )
        raise NotImplementedError(message)

    @staticmethod
    def _get_values_masks(array: ndarray) -> Tuple[bool, bool]:
        non_zero_values_mask = (array != 0)
        zero_values_mask = ~non_zero_values_mask
        return non_zero_values_mask, zero_values_mask
        

Как видим, это [абстрактный базовый класс](https://docs.python.org/3/library/abc.html). При 
инициализации мы задаём параметр `copy`, который нам пригодится в дальнейшем. Главное — у класса 
`BasePreprocessor` есть методы `fit()` и `transform()`. Первый отвечает за хранение характеристик 
данных, а второй за применение транформации с учётом самых характеристик. Есть также защищённый 
метод `_get_values_masks()`. О нём мы тоже поговорим чуть позже.

Рассмотрим теперь конкретную реализаицю Z-масштабирующего преобразователя.

In [ ]:
class ZScalingPreprocessor(BasePreprocessor):
    """Z-Scaling Preprocessor class for features standard scaling."""

    def __init__(self, copy: bool = True):
        super().__init__(copy)
        self.means: ndarray
        self.stds: ndarray

    def fit(self, x: ndarray) -> None:
        """Fit the preprocessor to the input x and computes the mean and
        standard deviation for each feature.

        :parameter x: The features to fit the preprocessor and compute the
        statistics.
            :type x: :class:`ndarray`
        """
        self.means = np.nanmean(x, axis=0)
        self.stds = np.nanstd(x, axis=0)

    def transform(self, x: ndarray) -> ndarray:
        """Transform the input features and standard scale the data according
        to the computed mean and standard deviation.

        :parameter x: Features to scale and transform.
            :type x: :class:`ndarray`

        :return: Standard scaled features.
            :rtype: :class:`ndarray`
        """
        if self.copy:
            x = x.copy()

        (nonzero_std_mask, zero_std_mask) = self._get_values_masks(self.stds)
        (nonzero_mean_mask, _) = self._get_values_masks(self.means)
        x[:, zero_std_mask] = 0
        
        x[:, nonzero_std_mask] = (
            x[:, nonzero_std_mask] - self.means[nonzero_mean_mask]
        ) / self.stds[nonzero_std_mask]
        return x

    def fit_transform(self, x) -> ndarray:
        """Fit and transform at the same time."""

        self.fit(x)

        transformed = self.transform(x)
        return transformed


Здесь стоит прояснить пару моментов...

***

## Задания

### Задание 1